# Projet P7 : Implémentez un modèle de scoring - Modèles

## Contexte

Vous êtes Data Scientist au sein d'une société financière, nommée "Prêt à dépenser", qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.

L’entreprise souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.)

<b> MISSION 1 : </b>

Construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.

Analyser les features qui contribuent le plus au modèle, d’une manière générale (feature importance globale) et au niveau d’un client (feature importance locale), afin, dans un soucis de transparence, de permettre à un chargé d’études de mieux comprendre le score attribué par le modèle.

Mettre en production le modèle de scoring de prédiction à l’aide d’une API et réaliser une interface de test de cette API.

<b> Approche MLOps : </b>

Afin de pouvoir faire évoluer régulièrement le modèle, mettre en œuvre une démarche de type MLOps d’automatisation et d’industrialisation de la gestion du cycle de vie du modèle (du tracking des expérimentations à l’analyse en production du data drift). 

Mettre en oeuvre au minimum les étapes orientées MLOps suivantes : 

- Dans le notebook d’entraînement des modèles, générer à l’aide de MLFlow un tracking d'expérimentations
- Lancer l’interface web 'UI MLFlow" d'affichage des résultats du tracking
- Réaliser avec MLFlow un stockage centralisé des modèles dans un “model registry”
- Tester le serving MLFlow
- Gérer le code avec le logiciel de version Git
- Partager le code sur Github pour assurer une intégration continue
- Utiliser Github Actions pour le déploiement continu et automatisé du code de l’API sur le cloud
- Concevoir des tests unitaires avec Pytest (ou Unittest) et les exécuter de manière automatisée lors du build réalisé par Github Actions

<b> Elaboration du modèle : </b>

Attention à deux points spécifiques au contexte métier : 

- Le déséquilibre entre le nombre de bons et de moins bons clients doit être pris en compte pour élaborer un modèle pertinent, avec une méthode au choix
- Le déséquilibre du coût métier entre un faux négatif (FN - mauvais client prédit bon client : donc crédit accordé et perte en capital) et un faux positif (FP - bon client prédit mauvais : donc refus crédit et manque à gagner en marge). Vous pourrez supposer, par exemple, que le coût d’un FN est dix fois supérieur au coût d’un FP. Vous créerez un score “métier” (minimisation du coût d’erreur de prédiction des FN et FP) pour comparer les modèles, afin de choisir le meilleur modèle et ses meilleurs hyperparamètres. Attention cette minimisation du coût métier doit passer par l’optimisation du seuil qui détermine, à partir d’une probabilité, la classe 0 ou 1 (un “predict” suppose un seuil à 0.5 qui n’est pas forcément l’optimum). En parallèle, maintenez pour comparaison et contrôle des mesures plus techniques, telles que l’AUC et l’accuracy.

Kernels Kaggle utilisés pour l’analyse exploratoire, la préparation des données et le feature engineering :
    
    - https://www.kaggle.com/code/willkoehrsen/start-here-a-gentle-introduction/notebook
    - https://www.kaggle.com/code/willkoehrsen/introduction-to-manual-feature-engineering
    - https://www.kaggle.com/code/willkoehrsen/introduction-to-manual-feature-engineering-p2
    - https://www.kaggle.com/code/willkoehrsen/introduction-to-feature-selection
    
    - https://www.kaggle.com/code/jsaguiar/lightgbm-with-simple-features/script --> pas utilisé pour le moment

## Etapes du projet :

Elaboration d'un modèle de prédiction sous forme d’une API qui permet de calculer la probabilité de défaut du client, ainsi que sa classe (accepté ou refusé), déployer l'API sur une plateforme Cloud.

- <b> Etape préliminaire : </b> Importation des données ( --> NoteBook1 : Preparation des données )
- <b> Etape 1 : </b> EDA et feature engineering sur la table principale application ( --> NoteBook1 ) 
- <b> Etape 2 : </b> Ajout des tables bureau et bureau_balance ( --> NoteBook1 )
- <b> Etape 3 : </b> Ajout des tables previous_application, POS_CASH_balance, installments_payments et credit_card_balance ( --> NoteBook1 )

- <b> Etape 4 : </b> EDA et Feature selection, feature engineering ( --> NoteBook1 )

- <b> Etape 5 : </b> Elaboration des modèles avec un tracking d'expérimentations (avec Cross-Validation et optimisation des hyperparamètres, via GridsearchCV ou équivalent)
- <b> Etape 6 : </b> Création de l'API (Notebook ou une application Streamlit pour réaliser en local l’interface de test de l’API)
- <b> Etape 7 : </b> Déploiement de l’API sur une plateforme Cloud (de préférence une solution gratuite)

## Imports


In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn methods
from sklearn.metrics import make_scorer, roc_curve, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Données déséquilibrées
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Models
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Outils MLOps
import mlflow
import mlflow.sklearn

from mlflow.models import infer_signature

## ETAPE 5 : Elaboration des modèles avec un tracking d'expérimentations

ETAPE 5
    
    1. Préparer les données
        - importer le dataset
        - scinder en train, validation, test
        - normaliser
    2. Création d'une fonction de coût métier
    3. Fonction pour implémenter un modèle et l'évaluer (tracking MLflow)
    4. Simulation des modèles

### 1. Préparer les données du modèle

#### -- Importer le dataset --

In [5]:
# Chargement du dataset train créé précédement, l'entrainement et la selection du modèle se fera uniquement sur ce dataset
data = pd.read_csv('./data/train_small.csv')
# data = pd.read_csv('./data/train_light.csv')

#### -- Scinder le dataset --

In [7]:
# Extract the ids
data_ids = data['SK_ID_CURR']
    
# Extract the labels for training
labels = data['TARGET']
    
# Remove the ids and target
X = data.drop(columns = ['SK_ID_CURR', 'TARGET'])

In [8]:
# Séparons nos données en entrainement et test
# Tout d'abord, nous divisons les données en jeu d'entraînement et jeu temporaire (ensemble de test + validation)
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, stratify=labels, random_state=30)

In [9]:
print("Dimensions dataset X_train, y_train : ", X_train.shape, y_train.shape)
print("Dimensions dataset X_test, y_test : ", X_test.shape, y_test.shape)

Dimensions dataset X_train, y_train :  (215257, 276) (215257,)
Dimensions dataset X_test, y_test :  (92254, 276) (92254,)


#### -- Normaliser les données --

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### 2. Création d'une fonction de coût métier

In [14]:
# TP : true positive, vrai positif : échantillons positifs prédits comme positifs.
# FP : false positive, faux positif : échantillons négatifs prédits comme positifs de façon erronée.
# TN : true negative, vrai négatif : échantillons négatifs prédits comme négatifs.
# FN : false negative, faux négatif : échantillons positifs prédits comme négatifs de façon erronée.
# Le rappel = TP / (TP + FN) (recall en anglais) est adapté pour minimiser les faux négatifs.

def cout_metier(y_true, y_pred):
    """Cette fonction calcule le coût métier à partir de la matrice de confusion : 10*FN + FP."""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel() # Je dois mettre y_proba ici ??? (pour prendre en compte le seuil)
    return 10 * fn + fp

cout_metier_scorer = make_scorer(cout_metier, greater_is_better=False)

def find_best_threshold(estimator, X, y):
    """Cette fonction trouve le seuil optimal en testant une gamme de seuils et en choisissant celui avec le score métier le plus bas."""
    thresholds = np.linspace(0, 1, 101)
    best_threshold, best_score = 0, float('inf')
    for threshold in thresholds:
        y_pred = (estimator.predict_proba(X)[:, 1] >= threshold).astype(int)
        score = cout_metier(y, y_pred)
        if score < best_score:
            best_threshold, best_score = threshold, score
    return best_threshold, best_score

### 3. Fonction pour implémenter un modèle et l'évaluer

In [16]:
# Faire une fonction pour définir un modèle, tester les hyper-paramètres par validation croisée, 
# entrainer le modele choisi et évaluer les indicateurs de performance du modèle

def modele_classification(estimator, parametres, folds, X1, y1, X2, y2, experiment_name="MLflow Classification Experiment"):
    """
    Renvoie le résultat du modèle avec les principaux indicateurs de performance pour la classification binaire
    0. X1, y1 = train set - X2, y2 = validation set
    1. Rééquilibrer les classes avec SMOTE
    2. Définition du modèle avec la liste de paramètres à tester
    3. Entrainement du modèle sur le train 
    4. Les meilleurs paramètres du modèle
    5. Le coût métier du modèle sur le train
    6. Calculer l'AUC du train "cross-validé"
    6. Prédictions pour X_test, scores sur le test et calcul des métriques
    7. Temps d'entrainement et predict du modèle
    8. MLflow tracking
    9. Stockage de la courbe ROC
    """
    
    # 1. Rééquilibrer les classes avec SMOTE
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('sampling', SMOTE()),
        ('classification', estimator)
    ])
    
    # 2. Définition du modèle avec la liste de paramètres à tester
    modele = GridSearchCV(
        estimator=pipeline,
        param_grid=parametres,
        cv=folds,
        scoring={'coût_métier': cout_metier_scorer, 'roc_auc': 'roc_auc', 'accuracy': 'accuracy'},
        refit='coût_métier'  # Optimiser selon notre coût métier
    )
    
    # 3. Entrainement du modèle sur le jeu d'entraînement
    modele.fit(X1, y1)
    
    # 4. Les meilleurs paramètres du modèle
    best_params = modele.best_params_
    print(f"Les meilleurs paramètres du modèle sont : {best_params}")
    
    # 5. Le coût métier du modèle sur le train
    best_cost = -modele.best_score_  # On a mis le scoring pour que plus petit soit mieux, donc le meilleur score est négatif
    print(f"Coût métier sur le train : {best_cost}")

    # 6. Calculer l'AUC du train "cross-validé"
    auc_train_scores = cross_val_score(pipeline, X1, y1, cv=folds, scoring='roc_auc')
    mean_auc_train = auc_train_scores.mean()
    std_auc_train = auc_train_scores.std()
    print(f"AUC train (cross-validé) : {mean_auc_train:.4f} ± {std_auc_train:.4f}")
    
    # 7. Trouver le seuil optimal
    best_threshold, best_threshold_score = find_best_threshold(modele.best_estimator_, X2, y2)
    print(f"Seuil optimal : {best_threshold} avec un score métier de : {best_threshold_score}")
    
    # Prédictions pour X_test et calcul des métriques pour identifier le meilleur modèle
    y_proba = modele.predict_proba(X2)[:, 1]
    y_pred = (y_proba >= best_threshold).astype(int)
    
    auc_test = roc_auc_score(y2, y_proba)
    cout_metier_test = cout_metier(y2, y_pred)
    f1_test = f1_score(y2, y_pred)
    accuracy_test = accuracy_score(y2, y_pred)

    print(f"Coût métier test : {cout_metier_test}")
    print(f"Score AUC test : {auc_test}")
    print(f"F1-score test : {f1_test}")
    print(f"Accuracy test : {accuracy_test}")

    # 8. Temps d'entrainement du modèle :
    temps_moyen_ajustement = modele.cv_results_['mean_fit_time'][modele.best_index_]
    print("Temps moyen d'ajustement du meilleur modèle (secondes):", temps_moyen_ajustement)
    # Temps Predict du modèle
    temps_moyen_inference = modele.cv_results_['mean_score_time'][modele.best_index_]
    print("Temps moyen d'inférence du meilleur modèle (secondes):", temps_moyen_inference)

    # 8. MLflow tracking
    mlflow.set_tracking_uri(uri="http://127.0.0.1:8080") # mlflow server --host 127.0.0.1 --port 8080 --> à lancer dans Anaconda Prompt
    mlflow.set_experiment(experiment_name)

    with mlflow.start_run():
        # Log the hyperparameters
        mlflow.log_params(best_params)

        # Log the metrics
        mlflow.log_metric("coût_métier_train", best_cost)
        mlflow.log_metric("AUC_train_cross_validé", mean_auc_train)
        mlflow.log_metric("coût_métier_test", cout_metier_test)
        mlflow.log_metric("auc_test", auc_test)
        mlflow.log_metric("f1_test", f1_test)
        mlflow.log_metric("accuracy_test", accuracy_test)
        mlflow.log_metric("best_threshold", best_threshold)
        mlflow.log_metric("mean_fit_time", temps_moyen_ajustement)
        mlflow.log_metric("mean_score_time", temps_moyen_inference)
        
        # 9. Stockage de la courbe ROC
        fpr, tpr, _ = roc_curve(y2, y_proba)
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc_test)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.savefig("roc_curve.png")
        plt.close()
        
        # Log the ROC curve as an artifact
        mlflow.log_artifact("roc_curve.png")
        
        # Set a tag for context
        mlflow.set_tag("Training Info", "Model training and evaluation for binary classification with custom cost function")

        # Infer the model signature
        signature = infer_signature(X1, modele.predict(X1))
        
        # Log the model
        mlflow.sklearn.log_model(
            sk_model=modele.best_estimator_,
            artifact_path="model",
            signature=signature,
            input_example=X1[:10],
            registered_model_name=experiment_name
        )

    return modele

### 4. Simulation des modèles

#### -- DummyClassifier --

In [51]:
# Définir les paramètres pour DummyClassifier
parametres = {
    'classification__strategy': ['most_frequent'] # ['most_frequent', 'stratified', 'uniform']
}

# Appeler la fonction avec un DummyClassifier
modele_Dummyclassif = modele_classification(
    estimator=DummyClassifier(),
    parametres=parametres,
    folds=5,
    X1=X_train,
    y1=y_train,
    X2=X_test,
    y2=y_test,
    experiment_name="Dummy Classifier"
)

Les meilleurs paramètres du modèle sont : {'classification__strategy': 'most_frequent'}
Coût métier sur le train : 34754.0
AUC train (cross-validé) : 0.5000 ± 0.0000
Seuil optimal : 0.01 avec un score métier de : 74480
Coût métier test : 74480
Score AUC test : 0.5
F1-score test : 0.0
Accuracy test : 0.9192663732737876
Temps moyen d'ajustement du meilleur modèle (secondes): 7.2784429550170895
Temps moyen d'inférence du meilleur modèle (secondes): 0.29339118003845216


Successfully registered model 'Dummy Classifier'.
2024/08/08 14:31:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Dummy Classifier, version 1
Created version '1' of model 'Dummy Classifier'.
2024/08/08 14:31:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run lyrical-midge-331 at: http://127.0.0.1:8080/#/experiments/231586947694154330/runs/f9300b22063e4538825ab6f9725531fe.
2024/08/08 14:31:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/231586947694154330.


#### -- Light GBM --

In [69]:
# Définir les paramètres pour LightGBM
parametres = {
    'classification__objective': ['binary'], 
    'classification__boosting_type': ['gbdt'],
    'classification__n_estimators': [100, 500, 1000], # [500, 1000]
    'classification__learning_rate': [0.001,0.01,0.1], # [0.001,0.01,0.1]
    'classification__class_weight': ['balanced'],
    'classification__verbose': [-1]
    }

modele_LGBM = modele_classification(
    estimator=lgb.LGBMClassifier(),
    parametres=parametres,
    folds=5,
    X1=X_train,
    y1=y_train,
    X2=X_test,
    y2=y_test,
    experiment_name="LightGBM Classification"
)

Les meilleurs paramètres du modèle sont : {'classification__boosting_type': 'gbdt', 'classification__class_weight': 'balanced', 'classification__learning_rate': 0.001, 'classification__n_estimators': 100, 'classification__objective': 'binary', 'classification__verbose': -1}
Coût métier sur le train : 32754.6
AUC train (cross-validé) : 0.7654 ± 0.0017
Seuil optimal : 0.48 avec un score métier de : 62431
Coût métier test : 62431
Score AUC test : 0.6535151052279731
F1-score test : 0.20540144339197114
Accuracy test : 0.6944739523489496
Temps moyen d'ajustement du meilleur modèle (secondes): 15.611072969436645
Temps moyen d'inférence du meilleur modèle (secondes): 0.5034257411956787


Successfully registered model 'LightGBM Classification'.
2024/08/08 22:44:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LightGBM Classification, version 1
Created version '1' of model 'LightGBM Classification'.
2024/08/08 22:44:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run polite-wren-23 at: http://127.0.0.1:8080/#/experiments/159117267029944621/runs/90eda313afbf4ba08f3a5a5c4d5c210c.
2024/08/08 22:44:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/159117267029944621.


In [ ]:
# -- SMOTE pour rééquilibrer les classes -- Finalement intégré directement dans le pipeline
# Appliquer SMOTE pour générer des sample de la classe minoritaire
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

#### -- RandomForestClassifier --

In [61]:
# Définir les paramètres
parametres = {
    'classification__n_estimators': [50, 100, 500], # [50, 100]
    'classification__max_depth': [5, 10, 20], # [3, 5, 10]
    'classification__min_samples_split': [30, 50, 100], # [2, 5, 10]
    'classification__class_weight': ['balanced']
}

modele_RFclassif = modele_classification(
    estimator=RandomForestClassifier(),
    parametres=parametres,
    folds=5,
    X1=X_train,
    y1=y_train,
    X2=X_test,
    y2=y_test,
    experiment_name="Random Forest Classifier"
)

Les meilleurs paramètres du modèle sont : {'classification__class_weight': 'balanced', 'classification__max_depth': 10, 'classification__min_samples_split': 50, 'classification__n_estimators': 50}
Coût métier sur le train : 29666.6
AUC train (cross-validé) : 0.7199 ± 0.0024
Seuil optimal : 0.34 avec un score métier de : 57775


2024/08/08 17:30:43 INFO mlflow.tracking.fluent: Experiment with name 'Random Forest Classifier' does not exist. Creating a new experiment.


Coût métier test : 57775
Score AUC test : 0.6894636749502427
F1-score test : 0.22282317399990254
Accuracy test : 0.6542155353697401
Temps moyen d'ajustement du meilleur modèle (secondes): 208.20021142959595
Temps moyen d'inférence du meilleur modèle (secondes): 1.1513511180877685


Successfully registered model 'Random Forest Classifier'.
2024/08/08 17:30:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Random Forest Classifier, version 1
Created version '1' of model 'Random Forest Classifier'.
2024/08/08 17:31:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run skittish-snipe-974 at: http://127.0.0.1:8080/#/experiments/902696018392126106/runs/c3355cb5bcde442ab7f263f136dfacb0.
2024/08/08 17:31:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/902696018392126106.


#### -- LogisticRegression --

In [65]:
# Définir les paramètres
parametres = {
    'classification__C': [0.01, 0.1, 1, 10, 100],
    'classification__class_weight': ['balanced']
}

modele_logRegression = modele_classification(
    estimator=LogisticRegression(),
    parametres=parametres,
    folds=5,
    X1=X_train,
    y1=y_train,
    X2=X_test,
    y2=y_test,
    experiment_name="Logistic Regression"
)

C:\Users\druar\anaconda3\envs\Formation_P7\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\druar\anaconda3\envs\Formation_P7\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

Les meilleurs paramètres du modèle sont : {'classification__C': 0.1, 'classification__class_weight': 'balanced'}
Coût métier sur le train : 22840.6


C:\Users\druar\anaconda3\envs\Formation_P7\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\druar\anaconda3\envs\Formation_P7\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

AUC train (cross-validé) : 0.7563 ± 0.0029
Seuil optimal : 0.54 avec un score métier de : 49680


2024/08/08 17:47:47 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression' does not exist. Creating a new experiment.


Coût métier test : 49680
Score AUC test : 0.7523182372667683
F1-score test : 0.2774678632064031
Accuracy test : 0.7416697378975438
Temps moyen d'ajustement du meilleur modèle (secondes): 16.164799165725707
Temps moyen d'inférence du meilleur modèle (secondes): 0.43104114532470705


Successfully registered model 'Logistic Regression'.
2024/08/08 17:47:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Logistic Regression, version 1
Created version '1' of model 'Logistic Regression'.
2024/08/08 17:48:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run omniscient-shrew-234 at: http://127.0.0.1:8080/#/experiments/787584366056825571/runs/618b5c2422fc434ea9d4c31e3f5bb308.
2024/08/08 17:48:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/787584366056825571.


#### -- DecisionTreeClassifier --

In [67]:
# Définir les paramètres
parametres = {
    'classification__max_depth': [5, 10, 20], 
    'classification__min_samples_split': [30, 50, 100],
    'classification__class_weight': ['balanced']
}

modele_logRegression = modele_classification(
    estimator=DecisionTreeClassifier(),
    
    parametres=parametres,
    folds=5,
    X1=X_train,
    y1=y_train,
    X2=X_test,
    y2=y_test,
    experiment_name="Decision Tree Classifier"
)


Les meilleurs paramètres du modèle sont : {'classification__class_weight': 'balanced', 'classification__max_depth': 5, 'classification__min_samples_split': 50}
Coût métier sur le train : 28811.6
AUC train (cross-validé) : 0.5400 ± 0.0012
Seuil optimal : 0.26 avec un score métier de : 62301


2024/08/08 19:51:22 INFO mlflow.tracking.fluent: Experiment with name 'Decision Tree Classifier' does not exist. Creating a new experiment.


Coût métier test : 62301
Score AUC test : 0.6365774062254123
F1-score test : 0.2004372389012749
Accuracy test : 0.5995945975242266
Temps moyen d'ajustement du meilleur modèle (secondes): 32.86524896621704
Temps moyen d'inférence du meilleur modèle (secondes): 0.3423666000366211


Successfully registered model 'Decision Tree Classifier'.
2024/08/08 19:51:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Decision Tree Classifier, version 1
Created version '1' of model 'Decision Tree Classifier'.
2024/08/08 19:51:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run bold-asp-485 at: http://127.0.0.1:8080/#/experiments/780877098331849492/runs/5b4df5ae621c4e6a94207c7dfcf01855.
2024/08/08 19:51:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/780877098331849492.
